# Exploratory Data Analysis

[Introduction]

## Imports

The following code imports the necessary libraries for this notebook:


In [62]:
import polars as pl
import pandas as pd
import plotly.express as px
import holidays

from datetime import datetime

## Dataset

The dataset used in this work contains information about the energy consumption, measured in kilowatt-hours (kWh), of 499 anonymized customers located in Spain. The dataset covers the full year of 2019, with observations recorded every hour. In addition to energy consumption, weather data are also provided, specifically the outside temperature in the region of each customer, with the same hourly resolution. Furthermore, each customer is assigned to one of 68 predefined customer profiles, such as private households, restaurants, or factories, allowing for segmentation and analysis based on these categories. 

The dataset is publicly available at [https://fordatis.fraunhofer.de/handle/fordatis/215](https://fordatis.fraunhofer.de/handle/fordatis/215).

The data is provided in three separate files: 
- `consumption.xlsx`, which contains the energy consumption data for the 499 customers. This file includes a `date` column with hourly timestamps and a column for each customer identifier, representing the energy consumption for that hour.
- `weather.xlsx`, which includes the hourly weather data (temperature) for the region of each customer. It also contains a `date` column with hourly timestamps and a column for each customer identifier, representing the temperature for that hour.
- `profiles.xlsx`, which contains information about the 68 predefined customer profiles. This file includes a column for the customer identifiers and another column specifying the associated activity, which defines the customer profile.

The first few rows of each file are displayed below to provide an overview of their structure:


In [63]:
consumption_df = pl.read_excel('../data/raw/consumption.xlsx')

print("First 5 rows of consumption dataset:")
consumption_df.head(5)

First 5 rows of consumption dataset:


date,5d6fcd1cf44b0324bc6b7254,5d6fcd1cf44b0324bc6b7257,5d6fcd1cf44b0324bc6b725a,5d6fcd1cf44b0324bc6b725d,5d6fcd1df44b0324bc6b7260,5d6fcd1df44b0324bc6b726b,5d6fcd1df44b0324bc6b7271,5d6fcd1df44b0324bc6b7274,5d6fcd1ef44b0324bc6b727a,5d6fcd1ef44b0324bc6b727c,5d6fcd1ef44b0324bc6b7289,5d6fcd1ef44b0324bc6b728b,5d6fcd1ff44b0324bc6b728d,5d6fcd1ff44b0324bc6b7293,5d6fcd1ff44b0324bc6b7296,5d6fcd1ff44b0324bc6b7299,5d6fcd1ff44b0324bc6b729b,5d6fcd1ff44b0324bc6b729e,5d6fcd20f44b0324bc6b72a4,5d6fcd20f44b0324bc6b72aa,5d6fcd20f44b0324bc6b72b0,5d6fcd20f44b0324bc6b72b6,5d6fcd21f44b0324bc6b72bc,5d6fcd21f44b0324bc6b72bf,5d6fcdddf44b0324bc6b815b,5d6fcdddf44b0324bc6b815c,5d6fcd21f44b0324bc6b72c5,5d6fcd22f44b0324bc6b72dd,5d6fcd23f44b0324bc6b72e7,5d6fcd23f44b0324bc6b72ed,5d6fcd23f44b0324bc6b72f3,5d6fcd23f44b0324bc6b72f6,5d6fcd23f44b0324bc6b72fa,5d6fcd24f44b0324bc6b72fd,5d6fcd24f44b0324bc6b7302,5d6fcd25f44b0324bc6b7313,…,5d6fcd79f44b0324bc6b79a8,5d6fcd79f44b0324bc6b79ae,5d6fcd79f44b0324bc6b79b1,5d6fcd7bf44b0324bc6b79c2,5d6fcd7bf44b0324bc6b79c1,5d6fcd7bf44b0324bc6b79c0,5d6fcd7af44b0324bc6b79bf,5d6fcd79f44b0324bc6b79b6,5d6fcd7af44b0324bc6b79be,5d6fcd7af44b0324bc6b79bd,5d6fcd7af44b0324bc6b79bc,5d6fcd7af44b0324bc6b79bb,5d6fcd7af44b0324bc6b79ba,5d6fcd7af44b0324bc6b79b9,5d6fcd79f44b0324bc6b79b5,5d6fcd7af44b0324bc6b79b8,5d6fcd79f44b0324bc6b79b7,5d6fcd7bf44b0324bc6b79c5,5d6fcd7bf44b0324bc6b79c7,5d6fcd7bf44b0324bc6b79c9,5d6fcd7bf44b0324bc6b79cc,5d6fcd7bf44b0324bc6b79cf,5d6fcd7bf44b0324bc6b79d1,5d6fcd7cf44b0324bc6b79d4,5d6fcd7cf44b0324bc6b79d7,5d6fcd7cf44b0324bc6b79dc,5d6fcd7cf44b0324bc6b79df,5d6fcd7cf44b0324bc6b79e4,5d6fcd7df44b0324bc6b79ed,5d6fcd7df44b0324bc6b79ef,5d6fcd7df44b0324bc6b79f1,5d6fcd7df44b0324bc6b79f4,5d6fcd7df44b0324bc6b79f7,5d6fcd7df44b0324bc6b79fd,5d6fcd7ef44b0324bc6b7a00,5d6fcd7ef44b0324bc6b7a02,5d6fcd7ef44b0324bc6b7a03
datetime[ms],f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2019-01-01 00:00:00,0.039,0.384,0.986,0.706,0.304,0.073,3.262,3.0,0.645,1.279,0.0,0.349,0.517,0.465,0.633,0.967,0.044,1.806,0.0,0.147,6.327,0.379,0.033,0.14,0.325,0.044,0.145,0.024,0.495,0.279,4.479,0.082,0.038,0.106,3.0,3,…,0.035,2.511,1.223,0.181,0.231,0.318,0.112,0.195,0.403,0.144,0.145,0.196,0.284,0.21,0.512,0.248,0.165,4.413,0.308,1.397,3.293,0.013,0.002,0.301,2.585,0.039,1.67,0.431,0.1,0.01,0.258,0.009,0.488,0.81,0.309,0.366,0.29
2019-01-01 01:00:00,0.269,0.051,0.846,2.21,0.673,0.047,3.252,4.0,0.746,0.851,0.0,0.347,0.589,0.171,0.635,0.952,0.11,1.706,0.0,0.145,4.237,0.409,0.036,0.172,0.322,0.044,0.142,0.614,0.492,0.385,4.262,0.095,0.156,0.106,2.0,3,…,0.036,2.299,1.293,0.182,0.207,0.286,0.101,0.216,0.435,0.23,0.208,0.177,0.29,0.186,0.832,0.236,0.167,5.587,0.183,1.075,1.365,0.033,0.003,0.287,2.506,0.425,1.609,0.447,0.025,0.009,0.232,0.009,0.447,0.753,0.381,0.308,0.369
2019-01-01 02:00:00,0.331,0.049,0.97,1.797,0.31,0.058,3.043,4.0,0.678,0.789,0.0,0.347,0.588,0.178,0.628,0.886,0.044,1.518,0.0,0.146,3.772,0.409,0.038,0.182,0.572,0.044,0.148,0.04,0.734,0.404,3.678,0.144,0.038,0.104,3.0,3,…,0.045,1.343,1.291,0.181,0.231,0.548,0.103,0.244,0.415,0.164,0.255,0.212,0.351,0.224,1.473,0.383,0.245,4.448,0.295,1.1,0.611,0.011,0.003,0.294,2.497,0.134,1.583,0.439,0.025,0.009,0.26,0.009,0.508,0.092,0.193,0.137,0.292
2019-01-01 03:00:00,0.093,0.049,0.803,1.01,0.545,0.059,2.96,4.0,0.644,0.637,0.0,0.349,0.432,0.477,0.627,1.013,0.058,1.757,0.0,0.148,3.553,0.411,0.04,0.242,0.322,0.045,0.212,0.055,0.434,0.227,3.844,0.082,0.174,0.685,2.0,2,…,0.026,2.703,1.284,0.18,0.224,0.369,0.103,0.217,0.409,0.163,0.171,0.173,0.29,0.216,0.532,0.213,0.165,1.564,0.296,1.909,0.171,0.033,0.003,0.339,2.696,0.153,1.492,0.913,0.023,0.009,0.252,0.009,0.511,0.032,0.124,0.201,0.358
2019-01-01 04:00:00,0.116,0.05,1.303,0.66,0.41,0.072,3.054,3.0,0.13,0.262,0.0,0.339,0.214,0.

In [64]:
weather_df = pl.read_excel('../data/raw/weather.xlsx')

print("First 5 rows of weather dataset:")
weather_df.head(5)

First 5 rows of weather dataset:


date,5d6fcd1cf44b0324bc6b7254,5d6fcd1cf44b0324bc6b7257,5d6fcd1cf44b0324bc6b725a,5d6fcd1cf44b0324bc6b725d,5d6fcd1df44b0324bc6b7260,5d6fcd1df44b0324bc6b726b,5d6fcd1df44b0324bc6b7271,5d6fcd1df44b0324bc6b7274,5d6fcd1ef44b0324bc6b727a,5d6fcd1ef44b0324bc6b727c,5d6fcd1ef44b0324bc6b7289,5d6fcd1ef44b0324bc6b728b,5d6fcd1ff44b0324bc6b728d,5d6fcd1ff44b0324bc6b7293,5d6fcd1ff44b0324bc6b7296,5d6fcd1ff44b0324bc6b7299,5d6fcd1ff44b0324bc6b729b,5d6fcd1ff44b0324bc6b729e,5d6fcd20f44b0324bc6b72a4,5d6fcd20f44b0324bc6b72aa,5d6fcd20f44b0324bc6b72b0,5d6fcd20f44b0324bc6b72b6,5d6fcd21f44b0324bc6b72bc,5d6fcd21f44b0324bc6b72bf,5d6fcdddf44b0324bc6b815b,5d6fcdddf44b0324bc6b815c,5d6fcd21f44b0324bc6b72c5,5d6fcd22f44b0324bc6b72dd,5d6fcd23f44b0324bc6b72e7,5d6fcd23f44b0324bc6b72ed,5d6fcd23f44b0324bc6b72f3,5d6fcd23f44b0324bc6b72f6,5d6fcd23f44b0324bc6b72fa,5d6fcd24f44b0324bc6b72fd,5d6fcd24f44b0324bc6b7302,5d6fcd25f44b0324bc6b7313,…,5d6fcd79f44b0324bc6b79a8,5d6fcd79f44b0324bc6b79ae,5d6fcd79f44b0324bc6b79b1,5d6fcd7bf44b0324bc6b79c2,5d6fcd7bf44b0324bc6b79c1,5d6fcd7bf44b0324bc6b79c0,5d6fcd7af44b0324bc6b79bf,5d6fcd79f44b0324bc6b79b6,5d6fcd7af44b0324bc6b79be,5d6fcd7af44b0324bc6b79bd,5d6fcd7af44b0324bc6b79bc,5d6fcd7af44b0324bc6b79bb,5d6fcd7af44b0324bc6b79ba,5d6fcd7af44b0324bc6b79b9,5d6fcd79f44b0324bc6b79b5,5d6fcd7af44b0324bc6b79b8,5d6fcd79f44b0324bc6b79b7,5d6fcd7bf44b0324bc6b79c5,5d6fcd7bf44b0324bc6b79c7,5d6fcd7bf44b0324bc6b79c9,5d6fcd7bf44b0324bc6b79cc,5d6fcd7bf44b0324bc6b79cf,5d6fcd7bf44b0324bc6b79d1,5d6fcd7cf44b0324bc6b79d4,5d6fcd7cf44b0324bc6b79d7,5d6fcd7cf44b0324bc6b79dc,5d6fcd7cf44b0324bc6b79df,5d6fcd7cf44b0324bc6b79e4,5d6fcd7df44b0324bc6b79ed,5d6fcd7df44b0324bc6b79ef,5d6fcd7df44b0324bc6b79f1,5d6fcd7df44b0324bc6b79f4,5d6fcd7df44b0324bc6b79f7,5d6fcd7df44b0324bc6b79fd,5d6fcd7ef44b0324bc6b7a00,5d6fcd7ef44b0324bc6b7a02,5d6fcd7ef44b0324bc6b7a03
datetime[ms],f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2019-01-01 00:00:00,10.31,10.35,1.65,1.7,11.53,10.76,10.57,0.43,1.66,10.49,8.98,10.33,1.71,8.5,10.75,1.69,1.81,8.55,10.72,-0.94,10.64,2.35,4.4,4.87,4.87,4.87,10.77,10.77,5.03,2.32,1.76,1.63,1.77,1.7,7.51,-0.42,…,11.18,0.14,9.18,9.48,9.48,9.48,9.48,9.48,9.48,9.48,9.48,9.48,9.48,9.48,9.48,9.48,9.48,11.83,12.1,1.61,0.08,10.71,10.6,1.68,10.71,10.33,4.53,10.4,9.53,9.94,3.19,3.14,3.2,2.63,1.51,1.51,1.51
2019-01-01 01:00:00,10.34,10.38,1.33,1.37,11.78,10.79,10.61,0.17,1.33,10.73,8.99,10.37,1.39,8.51,10.78,1.37,1.49,9.0,10.76,-1.2,10.68,1.92,3.96,4.64,4.64,4.64,10.8,10.8,4.77,1.73,1.44,1.31,1.44,1.37,7.77,-0.63,…,11.41,-0.16,9.18,9.48,9.48,9.48,9.48,9.48,9.48,9.48,9.48,9.48,9.48,9.48,9.48,9.48,9.48,11.95,12.24,1.3,-0.19,10.74,10.63,1.25,10.74,10.58,4.27,10.65,10.21,10.63,3.7,3.65,3.71,2.08,1.21,1.21,1.21
2019-01-01 02:00:00,10.36,10.4,1.01,1.05,12.03,10.82,10.64,-0.09,1.01,10.98,8.99,10.4,1.07,8.52,10.81,1.04,1.16,9.46,10.79,-1.46,10.71,1.5,3.52,4.4,4.4,4.4,10.83,10.83,4.51,1.14,1.11,0.99,1.12,1.05,8.03,-0.84,…,11.64,-0.45,9.17,9.47,9.47,9.47,9.47,9.47,9.47,9.47,9.47,9.47,9.47,9.47,9.47,9.47,9.47,12.07,12.37,0.99,-0.47,10.77,10.66,0.81,10.77,10.83,4.01,10.9,10.88,11.33,4.22,4.17,4.23,1.53,0.91,0.91,0.91
2019-01-01 03:00:00,10.39,10.43,0.69,0.73,12.28,10.84,10.68,-0.34,0.69,11.23,9.0,10.44,0.74,8.53,10.83,0.72,0.83,9.91,10.83,-1.71,10.75,1.08,3.09,4.17,4.17,4.17,10.85,10.85,4.25,0.55,0.79,0.67,0.79,0.73,8.3,-1.05,…,11.86,-0.74,9.16,9.46,9.46,9.46,9.46,9.46,9.46,9.46,9.46,9.46,9.46,9.46,9.46,9.46,9.46,12.19,12.5,0.68,-0.75,10.79,10.68,0.38,10.79,11.07,3.75,11.14,11.56,12.02,4.74,4.69,4.75,0.99,0.6,0.6,0.6
2019-01-01 04:00:00,10.27,10.31,0.47,0.51,12.13,10.73,10.57,-0.62,0.48,11.06,8.84,10.33,0.53,8.37,10.72,0.51,0.62,9.75,10.72,-1.88,10.64,0.91,3.14,4.24,4.24,4.24,10.74,10.74,4.29,-0.11,0.57,0.46,0.58,0.51,7.97,-1.39,…,11.69,-0.94,9.04,9.3

In [65]:
profiles_df = pl.read_excel('../data/raw/profiles.xlsx')

print("First 5 rows of profiles dataset:")
profiles_df.head(5)

First 5 rows of profiles dataset:


__UNNAMED__0,activity
str,str
"""5d6fcd1cf44b0324bc6b7254""","""Actividades de los hogares com…"
"""5d6fcd1cf44b0324bc6b7257""","""Actividades de los hogares com…"
"""5d6fcd1cf44b0324bc6b725a""","""Comercio al por menor de fruta…"
"""5d6fcd1cf44b0324bc6b725d""","""Actividades de los hogares com…"
"""5d6fcd1df44b0324bc6b7260""","""Actividades de los hogares com…"


## Data preparation

As can be seen when displaying the profiles, it appears that the first column containing the customer identifiers has no name, so it will be assigned the name 'customer' to facilitate the analysis.

In [66]:
profiles_df = profiles_df.rename({profiles_df.columns[0]: "customer"})

print("First 5 rows of profiles dataset:")
profiles_df.head(5)

First 5 rows of profiles dataset:


customer,activity
str,str
"""5d6fcd1cf44b0324bc6b7254""","""Actividades de los hogares com…"
"""5d6fcd1cf44b0324bc6b7257""","""Actividades de los hogares com…"
"""5d6fcd1cf44b0324bc6b725a""","""Comercio al por menor de fruta…"
"""5d6fcd1cf44b0324bc6b725d""","""Actividades de los hogares com…"
"""5d6fcd1df44b0324bc6b7260""","""Actividades de los hogares com…"


The first thing to check is that the column types are correct. In the case of the `consumption_df`, the `date` column should be of type `Datetime`, and the rest of the columns should be of type `Float64`. The same applies to the `weather_df`. The `profiles_df` should have the first column as `customer` (of type `String`) and the second column as `profile` (also of type `String`).

In [67]:
print("\nDistinct column types in consumption_df:")
print(set(consumption_df.dtypes))

print("\nDistinct column types in weather_df:")
print(set(weather_df.dtypes))

print("\nColumn types in profiles_df:")
print(profiles_df.dtypes)


Distinct column types in consumption_df:
{Float64, Datetime(time_unit='ms', time_zone=None), Int64}

Distinct column types in weather_df:
{Float64, Datetime(time_unit='ms', time_zone=None)}

Column types in profiles_df:
[String, String]


It seems that all the column types are correct, except for the `consumption_df`, where there is at least one column of the type `Int64`. It will be converted to `Float64` to ensure consistency across the dataset.

In [68]:
consumption_df = consumption_df.with_columns(
    [pl.col(c).cast(pl.Float64) for c in consumption_df.columns if c != "date"]
)

print("\nDistinct column types in consumption_df:")
print(set(consumption_df.dtypes))


Distinct column types in consumption_df:
{Float64, Datetime(time_unit='ms', time_zone=None)}


It is also necessary to check for the presence of null values in the dataset.

In [69]:
print("\nColumns with nulls in consumption_df:")
print([col for col in consumption_df.columns if consumption_df.select(pl.col(col).is_null().any()).item()])

print("\nColumns with nulls in weather_df:")
print([col for col in weather_df.columns if weather_df.select(pl.col(col).is_null().any()).item()])

print("\nColumns with nulls in profiles_df:")
print([col for col in profiles_df.columns if profiles_df.select(pl.col(col).is_null().any()).item()])


Columns with nulls in consumption_df:


[]

Columns with nulls in weather_df:
[]

Columns with nulls in profiles_df:
[]


There are no null values in the dataset, so no further action is needed in this regard.

It should also be verified how the daylight saving time changes are reflected in the dataset. In 2019, the first time change occurred on March 31, when the clocks were set forward by one hour at 2:00 AM. The second time change took place on October 27, when the clocks were set back by one hour at 3:00 AM.

In [70]:
march_31 = consumption_df.filter(
    (pl.col("date").dt.date() == pl.date(2019, 3, 31)) &
    (pl.col("date").dt.hour().is_between(1, 3))
).select("date")

print("March 31 timestamps:")
print(march_31)

oct_27 = consumption_df.filter(
    (pl.col("date").dt.date() == pl.date(2019, 10, 27)) &
    (pl.col("date").dt.hour().is_between(1, 3))
).select("date")

print("\nOctober 27 timestamps:")
print(oct_27)

March 31 timestamps:
shape: (2, 1)
┌─────────────────────┐
│ date                │
│ ---                 │
│ datetime[ms]        │
╞═════════════════════╡
│ 2019-03-31 01:00:00 │
│ 2019-03-31 03:00:00 │
└─────────────────────┘

October 27 timestamps:
shape: (4, 1)
┌─────────────────────┐
│ date                │
│ ---                 │
│ datetime[ms]        │
╞═════════════════════╡
│ 2019-10-27 01:00:00 │
│ 2019-10-27 02:00:00 │
│ 2019-10-27 02:00:00 │
│ 2019-10-27 03:00:00 │
└─────────────────────┘


As can be seen, there is no data for the hour 2:00 AM on March 31, so a row with the average consumption of the previous and next hour will be added to the dataset. In October 27, there are two rows for the hour 2:00 AM, so the average of these two rows will be taken to create a single row for the hour 2:00 AM. This ensures that the dataset has 24 rows for each day, maintaining the hourly resolution.

In [71]:
# Add row for 02:00 on March 31
prev_hour = consumption_df.filter(pl.col("date") == pl.datetime(2019, 3, 31, 1, 0, 0))
next_hour = consumption_df.filter(pl.col("date") == pl.datetime(2019, 3, 31, 3, 0, 0))

numeric_cols = [c for c in consumption_df.columns if c != "date"]
avg_values = (prev_hour.select(numeric_cols) + next_hour.select(numeric_cols)) / 2

avg_row = avg_values.with_columns(
    pl.lit(datetime(2019, 3, 31, 2, 0, 0), dtype=pl.Datetime("ms")).alias("date")
)

avg_row = avg_row.select(consumption_df.columns)

consumption_df = pl.concat([consumption_df, avg_row], how="vertical").sort("date")

march_31 = consumption_df.filter(
    (pl.col("date").dt.date() == pl.date(2019, 3, 31)) &
    (pl.col("date").dt.hour().is_between(1, 3))
).select("date")

print("March 31 timestamps:")
print(march_31)

# Calculate the average for October 27 at 2:00 AM 
oct_27_2am = consumption_df.filter(
    (pl.col("date").dt.date() == pl.date(2019, 10, 27)) &
    (pl.col("date").dt.hour() == 2)
)

numeric_cols = [c for c in consumption_df.columns if c != "date"]
avg_values = oct_27_2am.select(numeric_cols).mean()

dt_oct = datetime(2019, 10, 27, 2, 0, 0)
avg_row_oct = avg_values.with_columns(
    pl.lit(dt_oct, dtype=pl.Datetime("ms")).alias("date")
)

avg_row_oct = avg_row_oct.select(consumption_df.columns)

consumption_df = consumption_df.filter(~(
    (pl.col("date").dt.date() == pl.date(2019, 10, 27)) &
    (pl.col("date").dt.hour() == 2)
))
consumption_df = pl.concat([consumption_df, avg_row_oct], how="vertical").sort("date")

oct_27 = consumption_df.filter(
    (pl.col("date").dt.date() == pl.date(2019, 10, 27)) &
    (pl.col("date").dt.hour().is_between(1, 3))
).select("date")

print("\nOctober 27 timestamps:")
print(oct_27)

March 31 timestamps:
shape: (3, 1)
┌─────────────────────┐
│ date                │
│ ---                 │
│ datetime[ms]        │
╞═════════════════════╡
│ 2019-03-31 01:00:00 │
│ 2019-03-31 02:00:00 │
│ 2019-03-31 03:00:00 │
└─────────────────────┘

October 27 timestamps:
shape: (3, 1)
┌─────────────────────┐
│ date                │
│ ---                 │
│ datetime[ms]        │
╞═════════════════════╡
│ 2019-10-27 01:00:00 │
│ 2019-10-27 02:00:00 │
│ 2019-10-27 03:00:00 │
└─────────────────────┘


The same process will be applied to the `weather_df` dataframe to ensure consistency across both dataframes.

In [72]:
# Add row for 02:00 on March 31
prev_hour = weather_df.filter(pl.col("date") == pl.datetime(2019, 3, 31, 1, 0, 0))
next_hour = weather_df.filter(pl.col("date") == pl.datetime(2019, 3, 31, 3, 0, 0))

numeric_cols = [c for c in weather_df.columns if c != "date"]
avg_values = (prev_hour.select(numeric_cols) + next_hour.select(numeric_cols)) / 2

avg_row = avg_values.with_columns(
    pl.lit(datetime(2019, 3, 31, 2, 0, 0), dtype=pl.Datetime("ms")).alias("date")
)

avg_row = avg_row.select(weather_df.columns)

weather_df = pl.concat([weather_df, avg_row], how="vertical").sort("date")

march_31 = weather_df.filter(
    (pl.col("date").dt.date() == pl.date(2019, 3, 31)) &
    (pl.col("date").dt.hour().is_between(1, 3))
).select("date")

print("March 31 timestamps:")
print(march_31)

# Calculate the average for October 27 at 2:00 AM 
oct_27_2am = weather_df.filter(
    (pl.col("date").dt.date() == pl.date(2019, 10, 27)) &
    (pl.col("date").dt.hour() == 2)
)

numeric_cols = [c for c in weather_df.columns if c != "date"]
avg_values = oct_27_2am.select(numeric_cols).mean()

dt_oct = datetime(2019, 10, 27, 2, 0, 0)
avg_row_oct = avg_values.with_columns(
    pl.lit(dt_oct, dtype=pl.Datetime("ms")).alias("date")
)

avg_row_oct = avg_row_oct.select(weather_df.columns)

weather_df = weather_df.filter(~(
    (pl.col("date").dt.date() == pl.date(2019, 10, 27)) &
    (pl.col("date").dt.hour() == 2)
))
weather_df = pl.concat([weather_df, avg_row_oct], how="vertical").sort("date")

oct_27 = weather_df.filter(
    (pl.col("date").dt.date() == pl.date(2019, 10, 27)) &
    (pl.col("date").dt.hour().is_between(1, 3))
).select("date")

print("\nOctober 27 timestamps:")
print(oct_27)

March 31 timestamps:
shape: (3, 1)
┌─────────────────────┐
│ date                │
│ ---                 │
│ datetime[ms]        │
╞═════════════════════╡
│ 2019-03-31 01:00:00 │
│ 2019-03-31 02:00:00 │
│ 2019-03-31 03:00:00 │
└─────────────────────┘

October 27 timestamps:
shape: (3, 1)
┌─────────────────────┐
│ date                │
│ ---                 │
│ datetime[ms]        │
╞═════════════════════╡
│ 2019-10-27 01:00:00 │
│ 2019-10-27 02:00:00 │
│ 2019-10-27 03:00:00 │
└─────────────────────┘


At this point, it is more convenient to merge the three dataframes into a single one. The desired final structure is a dataframe with the following columns:

- `date`: the hourly timestamp of the observation (of type `Datetime`).
- `customer`: the customer identifier (of type `String`).
- `activity`: the customer profile (of type `String`).
- `consumption`: the energy consumption for that hour (of type `Float64`).
- `temperature`: the temperature for that hour (of type `Float64`).

First, an unpivot operation will be performed on the `consumption_df` and `weather_df` dataframes to transform them into a long format, where each row corresponds to a single observation for a specific customer at a specific time.

In [73]:
consumption_long = consumption_df.unpivot(
    index=["date"],
    variable_name="customer",
    value_name="consumption"
).with_columns(
    pl.col("consumption").cast(pl.Float64)
)

print("First 5 rows of consumption_long:")
print(consumption_long.head(5))

temperature_long = weather_df.unpivot(
    index=["date"],
    variable_name="customer",
    value_name="temperature"
).with_columns(
    pl.col("temperature").cast(pl.Float64)
)

print("First 5 rows of temperature_long:")
print(temperature_long.head(5))

First 5 rows of consumption_long:
shape: (5, 3)
┌─────────────────────┬──────────────────────────┬─────────────┐
│ date                ┆ customer                 ┆ consumption │
│ ---                 ┆ ---                      ┆ ---         │
│ datetime[ms]        ┆ str                      ┆ f64         │
╞═════════════════════╪══════════════════════════╪═════════════╡
│ 2019-01-01 00:00:00 ┆ 5d6fcd1cf44b0324bc6b7254 ┆ 0.039       │
│ 2019-01-01 01:00:00 ┆ 5d6fcd1cf44b0324bc6b7254 ┆ 0.269       │
│ 2019-01-01 02:00:00 ┆ 5d6fcd1cf44b0324bc6b7254 ┆ 0.331       │
│ 2019-01-01 03:00:00 ┆ 5d6fcd1cf44b0324bc6b7254 ┆ 0.093       │
│ 2019-01-01 04:00:00 ┆ 5d6fcd1cf44b0324bc6b7254 ┆ 0.116       │
└─────────────────────┴──────────────────────────┴─────────────┘
First 5 rows of temperature_long:
shape: (5, 3)
┌─────────────────────┬──────────────────────────┬─────────────┐
│ date                ┆ customer                 ┆ temperature │
│ ---                 ┆ ---                      ┆ ---     

Now that the dataframes are in the desired format, `consumption_long` and `temperature_long` can be merged into a single dataframe, using the `date` and `customer` columns as keys. The `activity` column will be added from the `profiles_df` dataframe by performing a left join on the `customer` column.

In [74]:
consumption_temperature = consumption_long.join(
    temperature_long,
    on=["date", "customer"],
    how="inner"
)

df = consumption_temperature.join(
    profiles_df,
    on="customer",
    how="left"
).select(
    "date",
    "customer",
    "activity",
    "consumption",
    "temperature"
)

print("First 5 rows of the merged dataframe:")
df.head(5)

First 5 rows of the merged dataframe:


date,customer,activity,consumption,temperature
datetime[ms],str,str,f64,f64
2019-01-01 00:00:00,"""5d6fcd1cf44b0324bc6b7254""","""Actividades de los hogares com…",0.039,10.31
2019-01-01 01:00:00,"""5d6fcd1cf44b0324bc6b7254""","""Actividades de los hogares com…",0.269,10.34
2019-01-01 02:00:00,"""5d6fcd1cf44b0324bc6b7254""","""Actividades de los hogares com…",0.331,10.36
2019-01-01 03:00:00,"""5d6fcd1cf44b0324bc6b7254""","""Actividades de los hogares com…",0.093,10.39
2019-01-01 04:00:00,"""5d6fcd1cf44b0324bc6b7254""","""Actividades de los hogares com…",0.116,10.27


## Exploratory Analysis

Once the data is prepared, an exploratory analysis will be performed to gain insights into the dataset.

### Consumption by customer activity

As it was mentioned at the beginning, the dataset contains information about 499 customers, ranging from households to businesses from various industrial sectors. Energy consumption in industries is likely to be somewhat higher than in households. To verify this, the customers will be grouped into two categories: "Household activities" and "Other activities". The average consumption for each group will be calculated and plotted over time to determine if there are significant differences in consumption patterns between the two groups.

In [75]:
household_activities = [
    "Actividades de los hogares como productores de servicios para uso propio",
    "Actividades de los hogares como productores de bienes para uso propio"
]

# Add a new column with group classification
df_group = df.with_columns(
    pl.when(pl.col("activity").is_in(household_activities))
      .then(pl.lit("Household activities"))
      .otherwise(pl.lit("Other activities"))
      .alias("group")
)

# Compute average consumption by group and date
df_avg = (
    df_group.group_by(["date", "group"])
      .agg(pl.mean("consumption").alias("avg_consumption"))
      .sort("date")
)

# Convert to pandas for Plotly
df_avg_pd = df_avg.to_pandas()

# Create line chart
fig = px.line(
    df_avg_pd,
    x="date",
    y="avg_consumption",
    color="group",
    title="Average consumption over time by activity group",
    labels={
        "date": "Date",
        "avg_consumption": "Average consumption (kWh)",
        "group": "Activity group"
    }
)

fig.show()

It can be observed that there is a consumption peak for the "Other activities" group during January, which may difficult to appreciate the differences between the two groups. To address this, the month of January will be excluded from the plot to better visualize the differences in consumption patterns throughout the rest of the year.

In [76]:
df_avg_feb = df_avg.filter(pl.col("date") >= datetime(2019, 2, 1))

df_avg_feb_pd = df_avg_feb.to_pandas()

fig = px.line(
    df_avg_feb_pd,
    x="date",
    y="avg_consumption",
    color="group",
    title="Average consumption over time by activity group (excluding January)",
    labels={
        "date": "Date",
        "avg_consumption": "Average consumption (kWh)",
        "group": "Activity group"
    }
)

fig.show()

This line chart clearly shows that the "Other activities" group consistently has higher average consumption compared to the "Household activities" group throughout the year. Due to this difference, the following analyses will be focused just on the 314 household customers, as they represent the majority of the dataset and are likely to have more similar consumption patterns.

In [77]:
df_households = df.filter(pl.col("activity").is_in(household_activities))

print("Number of rows in df_households:", df_households.height)

print("Number of distinct customers in df_households:", df_households.select(pl.col("customer").n_unique()).item())

print("First 5 rows of df_households:")
df_households.head(5)

Number of rows in df_households: 2750640
Number of distinct customers in df_households: 314
First 5 rows of df_households:


date,customer,activity,consumption,temperature
datetime[ms],str,str,f64,f64
2019-01-01 00:00:00,"""5d6fcd1cf44b0324bc6b7254""","""Actividades de los hogares com…",0.039,10.31
2019-01-01 01:00:00,"""5d6fcd1cf44b0324bc6b7254""","""Actividades de los hogares com…",0.269,10.34
2019-01-01 02:00:00,"""5d6fcd1cf44b0324bc6b7254""","""Actividades de los hogares com…",0.331,10.36
2019-01-01 03:00:00,"""5d6fcd1cf44b0324bc6b7254""","""Actividades de los hogares com…",0.093,10.39
2019-01-01 04:00:00,"""5d6fcd1cf44b0324bc6b7254""","""Actividades de los hogares com…",0.116,10.27


There are 2 types of activities classified as "Household activities", so it must be checked if there are significant differences in consumption patterns between them.

In [78]:
df_avg_activity = (
    df_households.group_by(["date", "activity"])
    .agg(pl.mean("consumption").alias("avg_consumption"))
    .sort("date")
)

df_avg_activity_pd = df_avg_activity.to_pandas()

fig = px.line(
    df_avg_activity_pd,
    x="date",
    y="avg_consumption",
    color="activity",
    title="Average consumption over time by household activity type",
    labels={
        "date": "Date",
        "avg_consumption": "Average consumption (kWh)",
        "activity": "Household activity"
    }
)

fig.show()

As it can be seen in the line chart, there consumption patterns of both types of household activities are quite similar. Therefore, for the sake of simplicity, both types of household activities will be treated as a single group in the subsequent analyses.

In [79]:
df_households = df_households.drop("activity")

print("First 5 rows of df_households:")
df_households.head(5)

First 5 rows of df_households:


date,customer,consumption,temperature
datetime[ms],str,f64,f64
2019-01-01 00:00:00,"""5d6fcd1cf44b0324bc6b7254""",0.039,10.31
2019-01-01 01:00:00,"""5d6fcd1cf44b0324bc6b7254""",0.269,10.34
2019-01-01 02:00:00,"""5d6fcd1cf44b0324bc6b7254""",0.331,10.36
2019-01-01 03:00:00,"""5d6fcd1cf44b0324bc6b7254""",0.093,10.39
2019-01-01 04:00:00,"""5d6fcd1cf44b0324bc6b7254""",0.116,10.27


### Consumption by day of the week

Another important aspect on which energy consumption depends is the day of the week. To analyze this, the average consumption for each day of the week will be calculated and plotted. The holidays will be treated as a separate category, as they can significantly affect consumption patterns. Since the exact location of each customer is unknown, only the national holidays in Spain will be considered.

In [80]:
# Spanish national holidays for 2019
es_holidays = holidays.Spain(years=[2019])
holiday_dates = set(es_holidays.keys())

# Create auxiliary column for date only
df_day_category = df_households.with_columns([
    pl.col("date").dt.date().alias("date_only"),
])

# Create day category column
df_day_category = df_day_category.with_columns(
    pl.when(pl.col("date_only").is_in([pd.to_datetime(d).date() for d in holiday_dates]))
      .then(pl.lit("holiday"))
      .otherwise(
          pl.col("date").dt.strftime("%A").str.to_lowercase()
      )
      .alias("day_category")
)

# Create hour column
df_day_category = df_day_category.with_columns(
    pl.col("date").dt.hour().alias("hour")
)

# Compute average consumption by day category and hour
df_avg_day_category = (
    df_day_category.group_by(["day_category", "hour"])
    .agg(pl.mean("consumption").alias("avg_consumption"))
    .sort(["day_category", "hour"])
)

# Convert to pandas for Plotly
df_avg_day_category_pd = df_avg_day_category.to_pandas()

# Define the order of days for better visualization
day_order = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday", "holiday"]
df_avg_day_category_pd["day_category"] = pd.Categorical(df_avg_day_category_pd["day_category"], categories=day_order, ordered=True)

# Create the plot
fig = px.line(
    df_avg_day_category_pd,
    x="hour",
    y="avg_consumption",
    color="day_category",
    category_orders={"day_category": day_order},
    title="Average hourly consumption by day of the week",
    labels={
        "hour": "Hour of day",
        "avg_consumption": "Average consumption (kWh)",
        "day_category": "Day category"
    }
)

fig.update_layout(
    xaxis=dict(dtick=1),
    legend=dict(title="Day category")
)

fig.show()

Some insights can be drawn from this line chart:

- Consumption on Tuesdays, Wednesdays, and Thursdays is quite similar, with only slight variations.
- Mondays follow a similar pattern to the midweek days, but with slightly lower consumption in the first 2 hours of the day.
- Something similar happens on Fridays, where the consumption is lower than on the midweek days in the last hours of the day.
- Saturdays have a lower consumption compared to weekdays, except for the early morning hours where the consumption is a bit higher.
- Consumption on Sundays is the lowest of all days, although it is similar to Saturdays during the early morning hours.
- Holidays have a consumption pattern similar to Sundays, but with slightly lower consumption during the first and last hours of the day.

Based on these observations, it could be possible to group the days into different categories to add this information as a feature in the predictive models. The proposed categories are as follows:
- **Monday**: Similar to midweek days but with lower consumption in the early hours.
- **Midweek**: Similar consumption patterns. 
- **Friday**: Similar to midweek days but with lower consumption in the late hours.
- **Saturday**: Lower consumption than weekdays, except for early morning hours.
- **Sunday and Holidays**: Lowest consumption overall, similar to Saturdays in the early morning hours.

Note that holidays could have been treated as a different category, but since consumption is quite similar to Sundays and there are only a few holidays in the dataset, it is more practical to group them with Sundays to avoid having a category with very few samples.